In [8]:
def start_signature():
    print("="*40)
    print("START OF NOTEBOOK — AT")
    print("="*40)

start_signature()


START OF NOTEBOOK — AT


In [9]:
import pandas as pd
import numpy as np


df_transfers = pd.read_csv(
r'C:\Users\arpitha_work\Downloads\TRU MSCDS\Sem 3\Graduate Project\MIMIC\mimic-iv-3.1\hosp\transfers.csv',
 parse_dates=["intime", "outtime"])
df_transfers.head()

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853.0,36904543,discharge,UNKNOWN,2180-05-07 17:21:27,NaT
3,10000032,22841357.0,34100253,discharge,UNKNOWN,2180-06-27 18:49:12,NaT
4,10000032,22841357.0,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12


In [10]:
# I want to know the unique number of patients and unique admissions 

unique_patients = df_transfers["subject_id"].nunique()
unique_admissions = df_transfers["hadm_id"].nunique()
unique_transfers = df_transfers["transfer_id"].nunique()
print("the no of unique patients :", unique_patients)
print(" the no of unique admissions :", unique_admissions)
print(" the no of unique transfers :", unique_transfers)

the no of unique patients : 364627
 the no of unique admissions : 546024
 the no of unique transfers : 2413581


In [11]:
df_transfers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2413581 entries, 0 to 2413580
Data columns (total 7 columns):
 #   Column       Dtype         
---  ------       -----         
 0   subject_id   int64         
 1   hadm_id      float64       
 2   transfer_id  int64         
 3   eventtype    object        
 4   careunit     object        
 5   intime       datetime64[ns]
 6   outtime      datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(2)
memory usage: 128.9+ MB


In [12]:

df_transfers = df_transfers.dropna(subset=["careunit", "intime", "outtime"]).copy()


df_transfers["dept_los_hours"] = (
    (df_transfers["outtime"] - df_transfers["intime"])
    .dt.total_seconds() / 3600
)


df_transfers = df_transfers[df_transfers["dept_los_hours"] > 0]


In [13]:
dept_basic = (
    df_transfers.groupby("careunit")
    .agg(
        num_stays   = ("transfer_id", "count"),
        num_patients= ("subject_id", "nunique"),
        los_mean    = ("dept_los_hours", "mean"),
        los_std     = ("dept_los_hours", "std")
    )
    .reset_index()
)

dept_basic.sort_values("num_stays", ascending=False).head(10)


,careunit,num_stays,num_patients,los_mean,los_std
6,Emergency Department,784303,342147,7.039268,6.712491
18,Medicine,191843,75822,69.081396,99.837228
7,Emergency Department Observation,101340,64017,4.742331,16.000225
5,Discharge Lounge,71241,50776,10.827407,7.034738
12,Med/Surg,56905,30433,66.872366,238.355833
19,Medicine/Cardiology,53891,26219,58.669751,78.900953
24,Neurology,49746,27304,64.780727,90.096014
41,Transplant,37903,11492,82.474241,110.275541
8,Hematology/Oncology,37354,9424,101.543491,164.375720
31,PACU,34958,28588,6.166163,12.030765


In [14]:
vol_threshold = dept_basic["num_stays"].quantile(0.75)
var_threshold = dept_basic["los_std"].quantile(0.50)

selected_depts = dept_basic[
    (dept_basic["num_stays"] >= vol_threshold) &
    (dept_basic["los_std"] >= var_threshold)
]

selected_depts.sort_values(
    ["num_stays", "los_std"], ascending=False
)


,careunit,num_stays,num_patients,los_mean,los_std
18,Medicine,191843,75822,69.081396,99.837228
12,Med/Surg,56905,30433,66.872366,238.355833
24,Neurology,49746,27304,64.780727,90.096014
41,Transplant,37903,11492,82.474241,110.275541
8,Hematology/Oncology,37354,9424,101.543491,164.375720


In [15]:
top_5_departments = selected_depts.head(5)
top_5_departments[["careunit", "num_stays", "los_mean", "los_std"]]


,careunit,num_stays,los_mean,los_std
8,Hematology/Oncology,37354,101.543491,164.375720
12,Med/Surg,56905,66.872366,238.355833
18,Medicine,191843,69.081396,99.837228
24,Neurology,49746,64.780727,90.096014
41,Transplant,37903,82.474241,110.275541


In [16]:
def end_signature():
    print("="*40)
    print("END OF NOTEBOOK — AT")
    print("="*40)

end_signature()


END OF NOTEBOOK — AT
